**MCMC(j)- TCI(k,l)**

In [5]:
import TensorCrossInterpolation as TCI




function f(v)
    j = v[1]
    k = v[2]
    l = v[3]
    
    NSq = N2^2
    
    term1 = cos(2*π*(j*k/NSq)) * sin(2*π*(j*l/NSq)) * exp(-j/N)
    term2 = sin(2*π*(j*k/NSq)) * cos(2*π*(j*l/NSq)) * exp(-j^2/NSq)

    return term1 + term2
end


function get_tci_val(j)
    localdims = fill(N2, 2)
    tolerance = 1e-8

    g(v) = f([j, v[1], v[2]]) #the method of fix one variable
    
    tci_2d, _, _ = TCI.crossinterpolate2(Float64, g, localdims; tolerance = tolerance )
    
    mps = tci_2d.sitetensors
    
    sumK = dropdims(sum(mps[1],dims=2),dims=2) #(r1 X N X r2) -> (r1 X 1 X r2) -> (r1 x r2)
    sumL = dropdims(sum(mps[2],dims=2),dims=2)

    resultMatrix = sumK * sumL #1x1 matrix
    
    return resultMatrix[1]

end


function mcmc_tci_sum(M)
    #define visit array
    vst = []

    #set init point
    j = rand(1:M)
    now = get_tci_val(j)
    memJ[j] = now
    push!(vst,now)
    

    for idx in 1:(M-1)
        jump = rand(-100:100)
        
        if (j + jump) < 1 || (j + jump) > M
            push!(vst,now)
            continue
        else
            j = j + jump

            #calculate proposal point
            prp = 0.0
            if memJ[j] == 0
                prp = get_tci_val(j)
                memJ[j] = prp
            else
                prp = memJ[j]
            end

            #MCMC
            accRatio = prp/now
            if rand() < accRatio
                #acc
                push!(vst,prp) 
                now = prp
            else
                #rej
                push!(vst,now) 
                j = j - jump #undo
            end
        end
    end

    return sum(vst)
end

function exact_sum(M)
    sum = 0.0
    
    for j in 1:M
        for k in 1:N2
            for l in 1:N2
                sum += f([j, k, l])
            end
        end
    end
    return sum / (M*(N2^2))
end

N = 10_000 #Num of j elements
N2 = 100 #Num of k,l elements
MLst = [10,100,1_000] #number of sampling
memJ = zeros(N)

 for M in MLst
    println("====M : $M====")
    epoch = 10
    acc = 0
    accSq = 0
    @time for i in 1:epoch

        mcmcTciSum = mcmc_tci_sum(M) / (M*(N2^2))

        acc += mcmcTciSum
        accSq += mcmcTciSum^2
    end

    stdev = sqrt((accSq / epoch - (acc / epoch)^2)/epoch)

    #println("MCMC-TCI stdev : $stdev" )

    extVal = exact_sum(M)
    println("MCMC-TCI result : $(acc/epoch)")
    println("Exact result : $extVal")
    println("stdev/sqrt(M) : $(stdev/sqrt(M))")

    if extVal - (stdev/sqrt(M)) <  (acc/epoch) || (acc/epoch) < extVal + (stdev/sqrt(M))
        println("Success :)")
    else
        println("Fail :(")
    end

end

====M : 10====
  1.812299 seconds (17.94 M allocations: 375.101 MiB, 4.57% gc time, 62.94% compilation time)
MCMC-TCI result : 0.3375866727097342
Exact result : 0.3293775199478671
stdev/sqrt(M) : 0.01711379273558438
Success :)
====M : 100====
  5.929236 seconds (176.60 M allocations: 3.143 GiB, 9.64% gc time)
MCMC-TCI result : 0.12861187226664014
Exact result : 0.21149490509535795
stdev/sqrt(M) : 0.010660388733611924
Success :)
====M : 1000====
 49.317526 seconds (1.49 G allocations: 26.306 GiB, 9.79% gc time)
MCMC-TCI result : 0.11869253286664787
Exact result : 0.021300376341638477
stdev/sqrt(M) : 0.002821102590978305
Success :)


**Proposal Distribution**
https://jduarte.physics.ucsd.edu/phys142/lectures/16_Path_Integral_MCMC.pdf (page 9)\
There's no need to use complicated proposal distribution. \
We can simply use vuniform distribution.